In [1]:
import pandas as pd
import openpyxl
hotelsdata = pd.read_excel(r"/Users/menimalina/Desktop/uni_due/coding/python/python-project/hotels/hotels.xlsx")
hotelsdata


,Unnamed: 0,hotel,rooms,price
0,0,hotel_1,13,273
1,1,hotel_2,18,92
2,2,hotel_3,12,141
3,3,hotel_4,18,157
4,4,hotel_5,7,298
...,...,...,...,...
395,395,hotel_396,5,212
396,396,hotel_397,12,68
397,397,hotel_398,14,102
398,398,hotel_399,16,284


In [8]:
print(5)

5
